## Read and Process ERSSTv5 data

This notebook contains code to pre-process the SST data from ERSSTv5: 
- regridding data to 5x5 grid
- calculating moving averages
- calculating long term trend for each grid cell using a 3rd order polynomail
- calculating grid-cell mean and standard deviations of detrended data
- calculating terciles (1/3 and 2/3 quantiles) of *detrended, standardized* data for each grid cell

Also saves date time series as .csv file and land mask as .nc file

In [1]:
import numpy as np
import xarray as xr
import xesmf as xe

In [2]:
out_res = 5
out_grid = xr.Dataset({'lat': (['lat'], np.arange(-90+out_res/2, 90+out_res/2, out_res)), 
                      'lon': (['lon'], np.arange(0+out_res/2, 360+out_res/2, out_res))})

out_grid

<xarray.Dataset>
Dimensions:  (lat: 36, lon: 72)
Coordinates:
  * lat      (lat) float64 -87.5 -82.5 -77.5 -72.5 -67.5 ... 72.5 77.5 82.5 87.5
  * lon      (lon) float64 2.5 7.5 12.5 17.5 22.5 ... 342.5 347.5 352.5 357.5
Data variables:
    *empty*

### Regridding

In [3]:
ds = xr.open_mfdataset("../input_data/ERSSTv5/sst.mnmean.nc")

In [4]:
regridder = xe.Regridder(ds, out_grid, method = "bilinear", periodic = True, 
                        ignore_degenerate=True)

In [5]:
ds_regrid = regridder(ds, keep_attrs=True)
ds_regrid.to_netcdf("../processed_data/ERSSTv5/sst_5x5.nc")
ds_regrid = xr.open_dataset("../processed_data/ERSSTv5/sst_5x5.nc")

In [6]:
## date time series
ds_regrid.time.to_dataframe().to_csv("../processed_data/ersstv5_date_timeseries.csv", index = False)

In [7]:
land_mask = xr.where(ds_regrid.isel(time = 0).sst.isnull(), 0, 1).rename("mask")
land_mask.to_netcdf("../processed_data/land_mask_5x5_ERSSTv5.nc")

### Moving averages

In [7]:
## rolling averages
ds_12 = ds_regrid.rolling(time = 12).mean(dim = "time")
ds_12.to_netcdf("../processed_data/ERSSTv5/sst_5x5_12month.nc")
ds_36 = ds_regrid.rolling(time = 36).mean(dim = "time")
ds_36.to_netcdf("../processed_data/ERSSTv5/sst_5x5_36month.nc")
ds_60 = ds_regrid.rolling(time = 60).mean(dim = "time")
ds_60.to_netcdf("../processed_data/ERSSTv5/sst_5x5_60month.nc")

### Trends

In [8]:
def calc_trend(ds, window, poly=3):
    nan_chunk = ds.isel(time = slice(0, window-1))
    N = len(ds.time)
    ds = ds.isel(time = slice(window-1, N))
    coeffs = ds.polyfit(dim = "time", deg = poly, skipna=True)
    trend_predict = xr.polyval(ds.time, coeffs).rename({"sst_polyfit_coefficients": "sst"})
    trend_predict = xr.concat([nan_chunk, trend_predict], dim = "time")
    
    return(trend_predict)

In [9]:
ds_12_trend = calc_trend(ds_12, window=12)
ds_12_trend.to_netcdf("../processed_data/ERSSTv5/sst_5x5_12month_trend_prediction.nc")
ds_36_trend = calc_trend(ds_36, window=36)
ds_36_trend.to_netcdf("../processed_data/ERSSTv5/sst_5x5_36month_trend_prediction.nc")
ds_60_trend = calc_trend(ds_60, window = 60)
ds_60_trend.to_netcdf("../processed_data/ERSSTv5/sst_5x5_60month_trend_prediction.nc")

### Mean and Standard Deviation

In [11]:
ds_12_mean = (ds_12 - ds_12_trend).mean(dim = "time")
ds_12_mean.to_netcdf("../processed_data/ERSSTv5/sst_5x5_12month_mean.nc")
ds_12_sd = (ds_12 - ds_12_trend).std(dim = "time")
ds_12_sd.to_netcdf("../processed_data/ERSSTv5/sst_5x5_12month_sd.nc")

ds_36_mean = (ds_36 - ds_36_trend).mean(dim = "time")
ds_36_mean.to_netcdf("../processed_data/ERSSTv5/sst_5x5_36month_mean.nc")
ds_36_sd = (ds_36 - ds_36_trend).std(dim = "time")
ds_36_sd.to_netcdf("../processed_data/ERSSTv5/sst_5x5_36month_sd.nc")

ds_60_mean = (ds_60 - ds_60_trend).mean(dim = "time")
ds_60_mean.to_netcdf("../processed_data/ERSSTv5/sst_5x5_60month_mean.nc")
ds_60_sd = (ds_60 - ds_60_trend).std(dim = "time")
ds_60_sd.to_netcdf("../processed_data/ERSSTv5/sst_5x5_60month_sd.nc")

/Users/fvdav22/Documents/research/multiyear-prediction/envs/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/fvdav22/Documents/research/multiyear-prediction/envs/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/fvdav22/Documents/research/multiyear-prediction/envs/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


### Quantiles

In [12]:
ds_12_quant = ((ds_12 - ds_12_trend - ds_12_mean)/ds_12_sd).quantile(q = np.array([1/3, 2/3]), dim = "time")
ds_12_quant.to_netcdf("../processed_data/ERSSTv5/sst_5x5_12month_quantiles.nc")

ds_36_quant = ((ds_36 - ds_36_trend - ds_36_mean)/ds_36_sd).quantile(q = np.array([1/3, 2/3]), dim = "time")
ds_36_quant.to_netcdf("../processed_data/ERSSTv5/sst_5x5_36month_quantiles.nc")

ds_60_quant = ((ds_60 - ds_60_trend - ds_60_mean)/ds_60_sd).quantile(q = np.array([1/3, 2/3]), dim = "time")
ds_60_quant.to_netcdf("../processed_data/ERSSTv5/sst_5x5_60month_quantiles.nc")

/Users/fvdav22/Documents/research/multiyear-prediction/envs/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1577: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,
